Configurando variável com a data e hora inicial

In [ ]:
from datetime import datetime
started = datetime.now()

Configurando ambiente spark

In [1]:
%%bash

# Instalação java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Instalação PySpark
pip install -q pyspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,533 kB]
Hit:13

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

Realizando download dos arquivos e salvando-os na camada Transient

In [3]:
#bibliotecas necessárias
import zipfile
import requests
from io import BytesIO
import os

In [5]:
#Variáveis gerais
link = "https://download.inep.gov.br/microdados/microdados_enem_2020.zip"
path_transient = "/content/transient/"
path_raw = "/content/raw/"
path_trusted = "/content/trusted/"
path_refined = "/content/refined/"

In [ ]:
#Realizando requisição na url do enem
filebytes = BytesIO( requests.get(link, verify=False).content )

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
#Descompactando o arquivo zip
myzip = zipfile.ZipFile(filebytes)
myzip.extractall(path_transient + "ENEM/")

Realizando a leitura full do arquivo na camada Transient

In [ ]:
df_enem = spark.read.options(header = 'True', delimiter=';', encoding = 'ISO-8859-1') \
            .csv(path_transient + "ENEM/DADOS/MICRODADOS_ENEM_2020.csv")

In [ ]:
df_enem.show()

Gravando o dataframe bruto como parquet em camada Raw

In [ ]:

df_enem.write.mode('overwrite').parquet(path_raw + "enem/")

Limpando a camada Transient

In [ ]:
!rm -rf '/content/transient/'

Realizando a leitura na camada Raw

In [ ]:
df_enem_raw = spark.read.load(path= path_raw + 'enem/*.parquet', format='parquet')

In [ ]:
df_enem_raw.show()

In [ ]:
df_enem_raw = df_enem_raw.select(
              "NU_INSCRICAO", 
              "TP_SEXO",
              "TP_COR_RACA",
              "TP_ESCOLA",
              "TP_ENSINO",
              "CO_MUNICIPIO_ESC",
              "NO_MUNICIPIO_ESC",
              "CO_UF_ESC",
              "SG_UF_ESC",
              "TP_LOCALIZACAO_ESC",
              "TP_SIT_FUNC_ESC",
              "NU_NOTA_REDACAO", 
              "NU_NOTA_CN", 
              "NU_NOTA_CH",
              "NU_NOTA_LC", 
              "NU_NOTA_MT", 
              "TP_PRESENCA_CN", 
              "TP_PRESENCA_CH", 
              "TP_PRESENCA_LC", 
              "TP_PRESENCA_MT"
               

) \

In [ ]:
df_enem_raw.printSchema()


Criando função para exibição de queries em SQL

In [ ]:
def fn_query(query, numero_linhas):
    return spark.sql(query).show(n = numero_linhas, truncate=False)

Criando view temporária para realizar os tratamentos de tipagem, criação de colunas, etc

In [ ]:
df_enem_raw.createOrReplaceTempView("VW_ENEM_RAW")

In [ ]:
#Visualizando a query
fn_query("""

    SELECT   

    BIGINT(NU_INSCRICAO) AS NU_INSCRICAO, 
    CASE 
      WHEN TP_SEXO = 'F' THEN 1
      WHEN TP_SEXO = 'M' THEN 2
      ELSE -1
    END CD_TP_SEXO,
    TP_SEXO AS SG_TP_SEXO, 
    CASE 
      WHEN TP_SEXO = 'F' THEN 'Feminino'
      WHEN TP_SEXO = 'M' THEN 'Masculino'
      ELSE 'Não Declarado'
    END NM_TP_SEXO,
    IFNULL(INT(TP_COR_RACA), -1) AS CD_TP_RACA,
    CASE 
      WHEN TP_COR_RACA = 0 THEN 'Não Declarado'
      WHEN TP_COR_RACA = 1 THEN 'Branca'
      WHEN TP_COR_RACA = 2 THEN 'Preta'
      WHEN TP_COR_RACA = 3 THEN 'Parda'
      WHEN TP_COR_RACA = 4 THEN 'Amarela'
      WHEN TP_COR_RACA = 5 THEN 'Indígena'
    END NM_COR_RACA,
    IFNULL(INT(TP_ESCOLA), -1) AS CD_TP_ESCOLA,
    CASE
      WHEN TP_ESCOLA = 1 THEN 'Não Respondeu'
      WHEN TP_ESCOLA = 2 THEN 'Pública'
      WHEN TP_ESCOLA = 3 THEN 'Privada'
      WHEN TP_ESCOLA = 4 THEN 'Exterior'
    END NM_TP_ESCOLA,
    IFNULL(INT(TP_ENSINO), -1) AS CD_TP_ENSINO,
    CASE 
      WHEN TP_ENSINO = 1 THEN 'Ensino Regular'
      WHEN TP_ENSINO = 2 THEN 'Educação Especial - Modalidade Substitutiva'
      WHEN TP_ENSINO = 3 THEN 'Educação de Jovens e Adultos'
      ELSE 'Não Respondeu'
    END NM_TP_ENSINO, 
    IFNULL(INT(CO_MUNICIPIO_ESC), -1) AS CD_MUNICIPIO_ESC,
    IFNULL(NO_MUNICIPIO_ESC, 'Não Respondeu') AS NM_MUNICIPIO_ESC,
    IFNULL(INT(CO_UF_ESC), -1) AS CD_UF_ESC,
    IFNULL(SG_UF_ESC, 'Não Respondeu') SG_UF_ESC,
    IFNULL(INT(TP_LOCALIZACAO_ESC), -1) AS CD_TP_LOCALIZACAO_ESC,
    CASE 
        WHEN TP_LOCALIZACAO_ESC = 1 THEN 'Urbana'
        WHEN TP_LOCALIZACAO_ESC = 2 THEN 'Rural'
        ELSE 'Não Respondeu'
    END NM_TP_LOCALIZACAO_ESC,
    IFNULL(INT(TP_SIT_FUNC_ESC), -1) AS CD_TP_SIT_FUNC_ESC,
    CASE 
         WHEN TP_SIT_FUNC_ESC = 1 THEN 'Em atividade'
         WHEN TP_SIT_FUNC_ESC = 2 THEN 'Paralisada'
         WHEN TP_SIT_FUNC_ESC = 3 THEN 'Extinta'
         WHEN TP_SIT_FUNC_ESC = 4 THEN 'Escola extinta em anos anteriores'
        ELSE 'Não Respondeu'
    END NM_TP_SIT_FUNC_ESC,
    FLOAT(NU_NOTA_REDACAO) NU_NOTA_REDACAO,
    FLOAT(NU_NOTA_CN) NU_NOTA_CN,
    FLOAT(NU_NOTA_CH) NU_NOTA_CH,
    FLOAT(NU_NOTA_LC) NU_NOTA_LC,
    FLOAT(NU_NOTA_MT) NU_NOTA_MT,
    INT(TP_PRESENCA_CN) AS CD_TP_PRESENCA_CN,
    INT(TP_PRESENCA_CH) AS CD_TP_PRESENCA_CH,
    INT(TP_PRESENCA_LC) AS CD_TP_PRESENCA_LC,
    INT(TP_PRESENCA_MT) AS CD_TP_PRESENCA_MT
    

     FROM VW_ENEM_RAW 

""", 15)

In [ ]:
#Atribuindo a query em um dataframe 
df_enem_raw_to_trusted = spark.sql("""

SELECT   

    BIGINT(NU_INSCRICAO) AS NU_INSCRICAO, 
    CASE 
      WHEN TP_SEXO = 'F' THEN 1
      WHEN TP_SEXO = 'M' THEN 2
      ELSE -1
    END CD_TP_SEXO,
    TP_SEXO AS SG_TP_SEXO, 
    CASE 
      WHEN TP_SEXO = 'F' THEN 'Feminino'
      WHEN TP_SEXO = 'M' THEN 'Masculino'
      ELSE 'Não Declarado'
    END NM_TP_SEXO,
    IFNULL(INT(TP_COR_RACA), -1) AS CD_TP_RACA,
    CASE 
      WHEN TP_COR_RACA = 0 THEN 'Não Declarado'
      WHEN TP_COR_RACA = 1 THEN 'Branca'
      WHEN TP_COR_RACA = 2 THEN 'Preta'
      WHEN TP_COR_RACA = 3 THEN 'Parda'
      WHEN TP_COR_RACA = 4 THEN 'Amarela'
      WHEN TP_COR_RACA = 5 THEN 'Indígena'
    END NM_COR_RACA,
    IFNULL(INT(TP_ESCOLA), -1) AS CD_TP_ESCOLA,
    CASE
      WHEN TP_ESCOLA = 1 THEN 'Não Respondeu'
      WHEN TP_ESCOLA = 2 THEN 'Pública'
      WHEN TP_ESCOLA = 3 THEN 'Privada'
      WHEN TP_ESCOLA = 4 THEN 'Exterior'
    END NM_TP_ESCOLA,
    IFNULL(INT(TP_ENSINO), -1) AS CD_TP_ENSINO,
    CASE 
      WHEN TP_ENSINO = 1 THEN 'Ensino Regular'
      WHEN TP_ENSINO = 2 THEN 'Educação Especial - Modalidade Substitutiva'
      WHEN TP_ENSINO = 3 THEN 'Educação de Jovens e Adultos'
      ELSE 'Não Respondeu'
    END NM_TP_ENSINO, 
    IFNULL(INT(CO_MUNICIPIO_ESC), -1) AS CD_MUNICIPIO_ESC,
    IFNULL(NO_MUNICIPIO_ESC, 'Não Respondeu') AS NM_MUNICIPIO_ESC,
    IFNULL(INT(CO_UF_ESC), -1) AS CD_UF_ESC,
    IFNULL(SG_UF_ESC, 'Não Respondeu') SG_UF_ESC,
    IFNULL(INT(TP_LOCALIZACAO_ESC), -1) AS CD_TP_LOCALIZACAO_ESC,
    CASE 
        WHEN TP_LOCALIZACAO_ESC = 1 THEN 'Urbana'
        WHEN TP_LOCALIZACAO_ESC = 2 THEN 'Rural'
        ELSE 'Não Respondeu'
    END NM_TP_LOCALIZACAO_ESC,
    IFNULL(INT(TP_SIT_FUNC_ESC), -1) AS CD_TP_SIT_FUNC_ESC,
    CASE 
         WHEN TP_SIT_FUNC_ESC = 1 THEN 'Em atividade'
         WHEN TP_SIT_FUNC_ESC = 2 THEN 'Paralisada'
         WHEN TP_SIT_FUNC_ESC = 3 THEN 'Extinta'
         WHEN TP_SIT_FUNC_ESC = 4 THEN 'Escola extinta em anos anteriores'
        ELSE 'Não Respondeu'
    END NM_TP_SIT_FUNC_ESC,
    FLOAT(NU_NOTA_REDACAO) NU_NOTA_REDACAO,
    FLOAT(NU_NOTA_CN) NU_NOTA_CN,
    FLOAT(NU_NOTA_CH) NU_NOTA_CH,
    FLOAT(NU_NOTA_LC) NU_NOTA_LC,
    FLOAT(NU_NOTA_MT) NU_NOTA_MT,
    INT(TP_PRESENCA_CN) AS CD_TP_PRESENCA_CN,
    INT(TP_PRESENCA_CH) AS CD_TP_PRESENCA_CH,
    INT(TP_PRESENCA_LC) AS CD_TP_PRESENCA_LC,
    INT(TP_PRESENCA_MT) AS CD_TP_PRESENCA_MT
    
     FROM VW_ENEM_RAW

 
""")

In [ ]:
df_enem_raw_to_trusted.printSchema()


Gravando o dataframe tratado na camada Truste

In [ ]:
df_enem_raw_to_trusted.write.mode('overwrite').parquet(path_trusted + "enem/")


Realizando leitura na camada Trusted e selecionando apenas as colunas necessárias para modelagem dimensional

In [ ]:
%%html
<center><iframe width="1000" height="640" src='https://dbdiagram.io/embed/62db19290d66c746553296b7'> </iframe></center>

FATO_ENEM

In [ ]:
df_fato_enem = spark.read.load(path = path_trusted + 'enem/*.parquet', format='parquet')

In [ ]:
df_fato_enem.show()

In [ ]:
df_fato_enem = df_fato_enem.select(

"CD_TP_SEXO",
"CD_TP_RACA",
"CD_TP_ESCOLA",
"CD_TP_ENSINO",
"CD_MUNICIPIO_ESC",
"CD_UF_ESC",
"CD_TP_LOCALIZACAO_ESC",
"CD_TP_SIT_FUNC_ESC",
"NU_INSCRICAO",
"NU_NOTA_REDACAO",
"NU_NOTA_CN",
"NU_NOTA_CH",
"NU_NOTA_LC",
"NU_NOTA_MT",
"CD_TP_PRESENCA_CN",
"CD_TP_PRESENCA_CH",
"CD_TP_PRESENCA_LC",
"CD_TP_PRESENCA_MT"

)

In [ ]:
df_fato_enem.write.mode('overwrite').parquet(path_refined + "fatos/fato_enem")


DM_SEXO

In [ ]:
df_dm_sexo = spark.read.load(path = path_trusted + 'enem/*.parquet', format='parquet')

In [ ]:
df_dm_sexo = df_dm_sexo.select("CD_TP_SEXO", "SG_TP_SEXO", "NM_TP_SEXO").distinct()

In [ ]:
df_dm_sexo.write.mode('overwrite').parquet(path_refined + "dimensoes/dm_sexo")

DM_ETNIA

In [ ]:
df_dm_etnia = spark.read.load(path = path_trusted + 'enem/*.parquet', format='parquet')

In [ ]:
df_dm_etnia = df_dm_etnia.select("CD_TP_RACA", "NM_COR_RACA").distinc

In [ ]:
df_dm_etnia.write.mode('overwrite').parquet(path_refined + "dimensoes/dm_etnia")

DM_TP_ESCOLA

In [ ]:
df_dm_tp_escola = spark.read.load(path = path_trusted + 'enem/*.parquet', format='parquet')

In [ ]:
df_dm_tp_escola = df_dm_tp_escola.select("CD_TP_ESCOLA","NM_TP_ESCOLA").distinct()

In [ ]:
df_dm_tp_escola.write.mode('overwrite').parquet(path_refined + "dimensoes/dm_tp_escola")

DM_TP_ENSINO

In [ ]:
df_dm_tp_ensino = spark.read.load(path = path_trusted + 'enem/*.parquet', format='parquet')

In [ ]:
df_dm_tp_ensino = df_dm_tp_ensino.select("CD_TP_ENSINO", "NM_TP_ENSINO").distinct()

In [ ]:
df_dm_tp_ensino.write.mode('overwrite').parquet(path_refined + "dimensoes/dm_tp_ensino")

DM_UF_ESC

In [ ]:
df_dm_tp_escola = spark.read.load(path = path_trusted + 'enem/*.parquet', format='parquet')

In [ ]:
df_dm_uf_escola = df_dm_uf_escola.select("CD_UF_ESC", "NM_MUNICIPIO_ESC").distinct()

In [ ]:
df_dm_municipio_esc.write.mode('overwrite').parquet(path_refined + 'dimensoes/dm_uf')

DM_MUNICIPIO_ESC

In [ ]:
df_dm_municipio_esc = spark.read.load(path = path_trusted + 'enem/*.parquet', format='parquet')

In [ ]:
df_dm_municipio_esc = df_dm_municipio_esc.select('CD_MUNICIPIO_ESC', 'NM_MUNICIPIO_ESC').distinct()

In [ ]:
df_dm_municipio_esc.write.mode('overwrite').parquet(path_refined + 'dimensoes/dm_municipio')

DM_LOCALIZACAO_ESC

In [ ]:
df_dm_localizacao_esc = spark.read.load(path = path_trusted + 'enem/*.parquet', format = 'parquet')

In [ ]:
df_dm_localizacao_esc = df_dm_municipio_esc.select('CD_TP_LOCALIZACAO_ESC', 'NM_TP_LOCALIZACAO_ESC').distinct()

In [ ]:
df_dm_localizacao_esc.write.mode('overwrite').parquet(path_refined + 'dimensones/dm_localizacao_esc')

DM_SITUACAO_ESCOLA

In [ ]:
df_dm_situacao_escola = spark.read.load(path = path_trusted + 'enem/*.parquet', format='parquet')

In [ ]:
df_dm_situacao_escola = df_dm_situacao_escola.select("CD_TP_SIT_FUNC_ESC", "NM_TP_SIT_FUNC_ESC").distinct()

In [ ]:
df_dm_situacao_escola.write.mode('overwrite').parquet(path_refined + "dimensoes/dm_situacao_esc")

DM_TP_PRESENCA

In [ ]:
df_dm_tp_presenca = spark.sql("""

SELECT 
      0 AS CD_TP_PRESENCA, 
      'Faltou à prova' AS NM_TP_PRESENCA
UNION

SELECT 
      1 AS CD_TP_PRESENCA, 
      'Presente na prova' AS NM_TP_PRESENCA

UNION

SELECT 
      2 AS CD_TP_PRESENCA, 
      'Eliminado na prova' AS NM_TP_PRESENCA

""")

In [ ]:
df_dm_tp_presenca.show()

In [ ]:
df_dm_tp_presenca.write.mode('overwrite').parquet(path_refined + "dimensoes/dm_tp_presenca")


Leitura na camada Refined

In [ ]:
df_dm_etnia = spark.read.load(path= path_refined + 'dimensoes/dm_etnia/*.parquet', format='parquet')
df_dm_localizacao_esc = spark.read.load(path= path_refined + 'dimensoes/dm_localizacao_esc/*.parquet', format='parquet')
df_dm_municipio = spark.read.load(path= path_refined + 'dimensoes/dm_municipio/*.parquet', format='parquet')
df_dm_sexo = spark.read.load(path= path_refined + 'dimensoes/dm_sexo/*.parquet', format='parquet')
df_dm_situacao_esc = spark.read.load(path= path_refined + 'dimensoes/dm_situacao_esc/*.parquet', format='parquet')
df_dm_tp_ensino = spark.read.load(path= path_refined + 'dimensoes/dm_tp_ensino/*.parquet', format='parquet')
df_dm_tp_escola = spark.read.load(path= path_refined + 'dimensoes/dm_tp_escola/*.parquet', format='parquet')
df_dm_tp_presenca = spark.read.load(path= path_refined + 'dimensoes/dm_tp_presenca/*.parquet', format='parquet')
df_dm_uf = spark.read.load(path= path_refined + 'dimensoes/dm_uf/*.parquet', format='parquet')
df_fato_enem = spark.read.load(path= path_refined + 'fatos/fato_enem/*.parquet', format='parquet')

In [ ]:
df_dm_etnia.createOrReplaceTempView("VW_DM_ETNIA")
df_dm_localizacao_esc.createOrReplaceTempView("VW_DM_LOCALIZACAO_ESC")
df_dm_municipio.createOrReplaceTempView("VW_DM_MUNICIPIO")
df_dm_sexo.createOrReplaceTempView("VW_DM_SEXO")
df_dm_situacao_esc.createOrReplaceTempView("VW_DM_SITUACAO_ESC")
df_dm_tp_ensino.createOrReplaceTempView("VW_DM_TP_ENSINO")
df_dm_tp_escola.createOrReplaceTempView("VW_DM_TP_ESCOLA")
df_dm_tp_presenca.createOrReplaceTempView("VW_DM_TP_PRESENCA")
df_dm_uf.createOrReplaceTempView("VW_DM_UF")
df_fato_enem.createOrReplaceTempView("VW_FATO_ENEM")

In [ ]:
#Média de notas
fn_query("""


SELECT 

TPE.NM_TP_ESCOLA,
TPENS.NM_TP_ENSINO,
UFES.SG_UF_ESC,
SITES.NM_TP_SIT_FUNC_ESC,      
AVG( F.NU_NOTA_REDACAO + F.NU_NOTA_CN + F.NU_NOTA_CH + F.NU_NOTA_LC + F.NU_NOTA_MT ) / 5 MEDIA

 FROM VW_FATO_ENEM F 

  INNER JOIN VW_DM_TP_ESCOLA TPE
    ON F.CD_TP_ESCOLA = TPE.CD_TP_ESCOLA

  INNER JOIN VW_DM_TP_ENSINO TPENS
    ON F.CD_TP_ENSINO = TPENS.CD_TP_ENSINO

  INNER JOIN VW_DM_UF UFES
    ON F.CD_UF_ESC = UFES.CD_UF_ESC

  INNER JOIN VW_DM_SITUACAO_ESC SITES
    ON F.CD_TP_SIT_FUNC_ESC = SITES.CD_TP_SIT_FUNC_ESC


  GROUP BY 

      TPE.NM_TP_ESCOLA,
      TPENS.NM_TP_ENSINO,
      UFES.SG_UF_ESC,
      SITES.NM_TP_SIT_FUNC_ESC

  ORDER BY MEDIA DESC


""", 1)

In [ ]:
#Aluno com maior nota dessa média
fn_query("""


SELECT 

F.NU_INSCRICAO, 
S.NM_TP_SEXO,
AVG( F.NU_NOTA_REDACAO + F.NU_NOTA_CN + F.NU_NOTA_CH + F.NU_NOTA_LC + F.NU_NOTA_MT ) / 5 MEDIA

FROM VW_FATO_ENEM F

INNER JOIN VW_DM_SEXO S
  ON F.CD_TP_SEXO = S.CD_TP_SEXO

GROUP BY NU_INSCRICAO, S.NM_TP_SEXO

ORDER BY MEDIA DESC


""", 1)

In [ ]:
#Média Geral
fn_query("""


SELECT 

AVG( F.NU_NOTA_REDACAO + F.NU_NOTA_CN + F.NU_NOTA_CH + F.NU_NOTA_LC + F.NU_NOTA_MT ) / 5 MEDIA

 FROM VW_FATO_ENEM F


""", 1)

In [ ]:
#Porcentagem de faltantes
fn_query("""


SELECT 

COUNT(F.NU_INSCRICAO) TOTAL_ALUNOS,

COUNT(
  CASE 
    WHEN CN.CD_TP_PRESENCA = 0 THEN 1
    WHEN CH.CD_TP_PRESENCA = 0 THEN 1
    WHEN LC.CD_TP_PRESENCA = 0 THEN 1
    WHEN MT.CD_TP_PRESENCA = 0 THEN 1
  END) TOTAL_AUSENTES_EM_UMA_DAS_PROVAS,

COUNT(
  CASE 
    WHEN CN.CD_TP_PRESENCA = 0 THEN 1
    WHEN CH.CD_TP_PRESENCA = 0 THEN 1
    WHEN LC.CD_TP_PRESENCA = 0 THEN 1
    WHEN MT.CD_TP_PRESENCA = 0 THEN 1
  END) / COUNT(F.NU_INSCRICAO) * 100 PERCENTUAL_AUSENTES_EM_UMA_DAS_PROVAS,
COUNT(
  CASE 
    WHEN 1=1
      AND  CN.CD_TP_PRESENCA = 0  
      AND  CH.CD_TP_PRESENCA = 0  
      AND  LC.CD_TP_PRESENCA = 0  
      AND  MT.CD_TP_PRESENCA = 0 
    THEN  1 
  END) TOTAL_AUSENTES_EM_TODAS_AS_PROVAS,
COUNT(
  CASE 
    WHEN 1=1
      AND  CN.CD_TP_PRESENCA = 0  
      AND  CH.CD_TP_PRESENCA = 0  
      AND  LC.CD_TP_PRESENCA = 0  
      AND  MT.CD_TP_PRESENCA = 0 
    THEN  1 
  END) / COUNT(F.NU_INSCRICAO) * 100 PERCENTUAL_AUSENTES_TODAS_AS_PROVAS

 FROM VW_FATO_ENEM F 

 INNER JOIN VW_DM_TP_PRESENCA CN
  ON F.CD_TP_PRESENCA_CN = CN.CD_TP_PRESENCA

 INNER JOIN VW_DM_TP_PRESENCA CH 
  ON F.CD_TP_PRESENCA_CH = CH.CD_TP_PRESENCA

 INNER JOIN VW_DM_TP_PRESENCA LC
  ON F.CD_TP_PRESENCA_LC = LC.CD_TP_PRESENCA

 INNER JOIN VW_DM_TP_PRESENCA MT 
  ON F.CD_TP_PRESENCA_MT = MT.CD_TP_PRESENCA

""", 1)

In [ ]:
#Total Inscrições
fn_query("""

SELECT COUNT(NU_INSCRICAO) AS TOTAL_INSCRITOS FROM VW_FATO_ENEM

""", 1)

In [ ]:
#Média por disciplina
fn_query("""


SELECT 

AVG(NU_NOTA_REDACAO) MEDIA_REDACAO,
AVG(NU_NOTA_CN) MEDIA_CIENCIAS_NATUREZA,
AVG(NU_NOTA_CH) MEDIA_CIENCIAS_HUMANAS,
AVG(NU_NOTA_LC) MEDIA_LINGUAGENS_CODIGOS,
AVG(NU_NOTA_MT) MEDIA_MATEMATICA


FROM VW_FATO_ENEM F


""", 1)

In [ ]:
#Média por sexo 
fn_query("""

SELECT 

SX.NM_TP_SEXO SEXO,
AVG( F.NU_NOTA_REDACAO + F.NU_NOTA_CN + F.NU_NOTA_CH + F.NU_NOTA_LC + F.NU_NOTA_MT ) / 5 MEDIA

FROM VW_FATO_ENEM F

INNER JOIN VW_DM_SEXO SX
  ON F.CD_TP_SEXO = SX.CD_TP_SEXO

GROUP BY SX.NM_TP_SEXO

ORDER BY MEDIA DESC

""", 2)

In [ ]:
#Média por etnia
fn_query("""

SELECT 

ETN.NM_COR_RACA ETNIA,
AVG( F.NU_NOTA_REDACAO + F.NU_NOTA_CN + F.NU_NOTA_CH + F.NU_NOTA_LC + F.NU_NOTA_MT ) / 5 MEDIA

FROM VW_FATO_ENEM F

INNER JOIN VW_DM_ETNIA ETN
  ON F.CD_TP_RACA = ETN.CD_TP_RACA

GROUP BY 
  ETN.NM_COR_RACA 

ORDER BY MEDIA DESC


""", 10)

In [ ]:
#tempo de execução do processo 
finished = datetime.now()
print(finished-started)